In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import torch
import datetime
from keras.preprocessing.sequence import pad_sequences
from keras.layers import Embedding, Input, LSTM
from keras.models import Sequential, Model,load_model
from keras.layers import Activation, Dense, Dropout, Embedding, Flatten, Input, Convolution1D, MaxPooling1D, GlobalMaxPooling1D
import numpy as np
from torch.utils.data import TensorDataset
from torch.utils.data import DataLoader
from torch import optim
from sklearn.preprocessing import MinMaxScaler
import torch.nn as nn
import datetime

Using TensorFlow backend.


In [ ]:
data=pd.read_csv("train.csv")

D:\Anaconda\envs\tensorflow\lib\site-packages\IPython\core\interactiveshell.py:2785: DtypeWarning: Columns (5) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [3]:
store_data=pd.read_csv("stores.csv")
items_data=pd.read_csv("items.csv")
transactions_data=pd.read_csv("transactions.csv")
holiday_data=pd.read_csv("holidays_events.csv")

In [4]:
reg_dict = {'El Oro': 'Coast','Esmeraldas': 'Coast','Santo Domingo de los Tsachilas':'Coast','Guayas':'Coast','Los Rios':'Coast'
           ,'Manabi':'Coast','Santa Elena':'Coast',
           'Azuay':'Sierra','Bolivar':'Sierra','Cotopaxi':'Sierra','Chimborazo':'Sierra','Imbabura':'Sierra'
            ,'Loja':'Sierra','Pichincha':'Sierra','Tungurahua':'Sierra','Pastaza':'Amazon'}

In [5]:
store_data['Geographic Region'] = store_data['state'].map(reg_dict)

In [6]:
store_data.drop(["city","state","type","cluster"],inplace=True,axis=1)

In [7]:
items_data.drop("class",axis=1,inplace=True)

### Stores

In [8]:
merged_data=data.merge(store_data,how="left",on="store_nbr")

In [9]:
agg_data=pd.DataFrame({'unit_sales' : merged_data.groupby(["store_nbr","date"])["unit_sales"].sum(),
                       'region':merged_data.groupby(["store_nbr","date"])["Geographic Region"].max()}).reset_index()

In [10]:
agg_data=agg_data.merge(transactions_data,how="left",on=["date","store_nbr"])

In [11]:
scaler = MinMaxScaler()
agg_data["unit_sales"]=scaler.fit_transform(np.array(agg_data.unit_sales).reshape(-1,1))
scaler1 = MinMaxScaler()
agg_data["transactions"]=scaler.fit_transform(np.array(agg_data.transactions).reshape(-1,1))

In [12]:
final_agg_data=pd.DataFrame()
for i in agg_data.store_nbr.unique():
    temp_data=agg_data[agg_data.store_nbr==i].reset_index(drop=True)
    for i in range(1,8):
        temp_data["t"+str(i)]=temp_data["unit_sales"].shift(i)
    temp_data.drop(temp_data[temp_data.isnull().any(axis=1)].index,inplace=True)
    final_agg_data=pd.concat([final_agg_data,temp_data])

In [13]:
final_agg_data.reset_index(inplace=True,drop=True)

In [14]:
final_agg_data["date"]=pd.to_datetime(final_agg_data.date)

In [15]:
final_agg_data["weekday"]=[i.weekday() for i in final_agg_data.date]
final_agg_data["month"]=[i.month for i in final_agg_data.date]

In [16]:
final_agg_data=pd.concat([pd.get_dummies(final_agg_data[['store_nbr', 'region','weekday', 'month']].astype(str)),
          final_agg_data[['date','transactions', 't1', 't2', 't3','t4', 't5', 't6', 't7','unit_sales']]],axis=1)

In [17]:
final_agg_data.to_csv("stores_agg.csv",index=None)

In [39]:
final_agg_data.drop("date",inplace=True,axis=1)

### Items

In [18]:
merged_data_items=data.merge(items_data,how="left",on="item_nbr")

In [19]:
agg_data_items=pd.DataFrame({'unit_sales' : merged_data_items.groupby(["date","family"])["unit_sales"].sum(),
                       'perishable':merged_data_items.groupby(["date","family"])["perishable"].mean()}).reset_index()

In [20]:
scaler2 = MinMaxScaler()
agg_data_items["unit_sales"]=scaler2.fit_transform(np.array(agg_data_items.unit_sales).reshape(-1,1))

In [21]:
final_agg_data_items=pd.DataFrame()
for i in agg_data_items.family.unique():
    temp_data=agg_data_items[agg_data_items.family==i].reset_index(drop=True)
    for i in range(1,8):
        temp_data["t"+str(i)]=temp_data["unit_sales"].shift(i)
    temp_data.drop(temp_data[temp_data.isnull().any(axis=1)].index,inplace=True)
    final_agg_data_items=pd.concat([final_agg_data_items,temp_data])

In [22]:
final_agg_data_items.reset_index(inplace=True,drop=True)

In [23]:
final_agg_data_items["date"]=pd.to_datetime(final_agg_data_items.date)

In [24]:
final_agg_data_items["weekday"]=[i.weekday() for i in final_agg_data_items.date]
final_agg_data_items["month"]=[i.month for i in final_agg_data_items.date]

In [25]:
final_agg_data_items=pd.concat([pd.get_dummies(final_agg_data_items[['family','weekday', 'month']].astype(str)),
          final_agg_data_items[['date','t1', 't2', 't3','t4', 't5', 't6', 't7','unit_sales']]],axis=1)

In [28]:
final_agg_data_items.to_csv("items_agg.csv",index=None)

In [14]:
final_agg_data_items.drop("date",inplace=True,axis=1)

### Final Model

In [9]:
agg_data_final=data.merge(store_data,how="left",on="store_nbr")

In [10]:
agg_data_final=agg_data_final.merge(items_data,on="item_nbr",how="left")

In [11]:
temp_unit_sales=agg_data_final.groupby(["date","store_nbr","family"])["unit_sales"].sum()

In [12]:
temp_perishable=agg_data_final.groupby(["date","store_nbr","family"])["perishable"].mean()

In [14]:
agg_data_final1=pd.DataFrame({'unit_sales' : temp_unit_sales,
                       'perishable': temp_perishable}).reset_index()

In [18]:
agg_data_final1=agg_data_final1.merge(store_data,on="store_nbr",how="left")

In [20]:
agg_data_final1["date"]=pd.to_datetime(agg_data_final1.date)

In [21]:
agg_data_final1["weekday"]=[i.weekday() for i in agg_data_final1.date]
agg_data_final1["month"]=[i.month for i in agg_data_final1.date]

In [23]:
agg_data_final1.to_csv("agg_final_data.csv",index=False)